In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk

import torch.nn.functional as F
import torch

In [3]:
label_ts = Path('/home/gridsan/nchutisilp/datasets/nnUNet_Datasets/nnUNet_raw/Dataset302_Calcium_OCTv2/labelsTs')

In [2]:
pretraining_suffixes = ['finetuned_from_scratch_test', 'finetuned_with_CLIP_test',
                        # 'finetuned_with_Genesis_test', 
                        'finetuned_with_LaW_test']
presentation_names = ['nnUNet 3D/160x128x32', 'nnUNet 3D/160x128x32 (CLIP Pre-&Post-IVL)', 
                      # 'nnUNet 3D/160x128x32 (Genesis Unannotated OCT)', 
                      'nnUNet 3D/160x128x32 (LaW OCT)']


In [4]:
label_names = list(label_path.name for label_path in label_ts.glob("*.nii.gz"))

batched_labels = []
for label_path in label_ts.glob("*.nii.gz"):
    # Read label to a numpy array
    label = sitk.ReadImage(label_path)
    label = sitk.GetArrayFromImage(label)
    
    # Convert a numpy array label to a Tensor
    label = torch.Tensor(label)
    label = label.long()
    label = label.unsqueeze(0)

    batched_labels.append(label)

# BxDxHxW
batched_labels = torch.vstack(batched_labels)

In [85]:
# root_path = Path('/home/gridsan/nchutisilp/datasets/nnUNet_Datasets/nnUNet_results/Dataset302_Calcium_OCTv2/nnUNetTrainer__nnUNetPlans__3d_32x160x128_b10/')
# losses = dict()
# for presentation_name, pretraining_suffix in zip(presentation_names, pretraining_suffixes):
#     losses[presentation_name] = []
#     print(presentation_name, end=" ")
#     for fold in range(3):
#         print(fold , end=", ")

#         prediction_folder = root_path / f'fold_{fold}_{pretraining_suffix}'

#         batched_log_probabilities = []
#         for label_name in label_names:
#             # Read prediction to a numpy array
#             prediction = np.load(prediction_folder / label_name.replace(".nii.gz", ".npz"))
#             prediction = prediction['probabilities']

#             # Convert a numpy array probabilities into a log probability Tensor
#             prediction = torch.Tensor(prediction)
#             prediction = torch.log(prediction)
#             prediction = prediction.unsqueeze(0)

#             batched_log_probabilities.append(prediction)

#         # BxCxDxHxW
#         batched_log_probabilities = torch.vstack(batched_log_probabilities)

#         ce_loss = F.nll_loss(batched_log_probabilities, batched_labels)
#         losses[presentation_name].append(ce_loss.item())
#     print("Done!")
# torch.save(losses, 'nnUNetTrainer_CELosses_woPre_CLIP_LAW.pt')

nnUNet 3D/160x128x32 0, 1, 2, Done!
nnUNet 3D/160x128x32 (CLIP Pre-&Post-IVL) 0, 1, 2, Done!
nnUNet 3D/160x128x32 (LaW OCT) 0, 1, 2, Done!


In [5]:
for scale in range(3,6):
    print("Scale", scale)
    root_path = Path(f'/home/gridsan/nchutisilp/datasets/nnUNet_Datasets/nnUNet_results/Dataset302_Calcium_OCTv2/nnUNetTrainerScaleAnalysis{scale}__nnUNetPlans__3d_32x160x128_b10/')

    losses = dict()
    for presentation_name, pretraining_suffix in zip(presentation_names, pretraining_suffixes):
        losses[presentation_name] = []
        print(presentation_name, end=" ")
        for fold in range(3):
            print(fold , end=", ")

            prediction_folder = root_path / f'fold_{fold}_{pretraining_suffix}'

            batched_log_probabilities = []
            for label_name in label_names:
                # Read prediction to a numpy array
                prediction = np.load(prediction_folder / label_name.replace(".nii.gz", ".npz"))
                prediction = prediction['probabilities']

                # Convert a numpy array probabilities into a log probability Tensor
                prediction = torch.Tensor(prediction)
                prediction = torch.log(prediction)
                prediction = prediction.unsqueeze(0)

                batched_log_probabilities.append(prediction)

            # BxCxDxHxW
            batched_log_probabilities = torch.vstack(batched_log_probabilities)

            ce_loss = F.nll_loss(batched_log_probabilities, batched_labels)
            losses[presentation_name].append(ce_loss.item())
        print("Done!")
    torch.save(losses, f'nnUNetTrainerScaleAnalysis{scale}_CELosses_woPre_CLIP_LAW.pt')

Scale 3
nnUNet 3D/160x128x32 0, 1, 2, Done!
nnUNet 3D/160x128x32 (CLIP Pre-&Post-IVL) 0, 1, 2, Done!
nnUNet 3D/160x128x32 (LaW OCT) 0, 1, 2, Done!
Scale 4
nnUNet 3D/160x128x32 0, 1, 2, Done!
nnUNet 3D/160x128x32 (CLIP Pre-&Post-IVL) 0, 1, 2, Done!
nnUNet 3D/160x128x32 (LaW OCT) 0, 1, 2, Done!
Scale 5
nnUNet 3D/160x128x32 0, 1, 2, Done!
nnUNet 3D/160x128x32 (CLIP Pre-&Post-IVL) 0, 1, 2, Done!
nnUNet 3D/160x128x32 (LaW OCT) 0, 1, 2, Done!


In [7]:
losses = torch.load('nnUNetTrainer_CELosses_woPre_CLIP_LAW.pt')
losses

{'nnUNet 3D/160x128x32': [0.05369274690747261,
  0.0596805214881897,
  0.05074412375688553],
 'nnUNet 3D/160x128x32 (CLIP Pre-&Post-IVL)': [0.0556299164891243,
  0.056006330996751785,
  0.04590117186307907],
 'nnUNet 3D/160x128x32 (LaW OCT)': [0.04940780624747276,
  0.04896751418709755,
  0.0490996427834034]}

In [11]:
for scale in range(2,6):
    print(scale)
    losses = torch.load(f'nnUNetTrainerScaleAnalysis{scale}_CELosses_woPre_CLIP_LAW.pt')
    print(losses)
    print()

2
{'nnUNet 3D/160x128x32': [0.10112794488668442, 0.0981173887848854, 0.12073627859354019], 'nnUNet 3D/160x128x32 (CLIP Pre-&Post-IVL)': [0.10181018710136414, 0.09586725383996964, 0.12667453289031982], 'nnUNet 3D/160x128x32 (LaW OCT)': [0.07728803902864456, 0.07829064130783081, 0.090856172144413]}

3
{'nnUNet 3D/160x128x32': [0.15814074873924255, 0.0674314796924591, 0.09355108439922333], 'nnUNet 3D/160x128x32 (CLIP Pre-&Post-IVL)': [0.055596042424440384, 0.0710187628865242, 0.0884246826171875], 'nnUNet 3D/160x128x32 (LaW OCT)': [0.061689358204603195, 0.063583143055439, 0.0835082158446312]}

4
{'nnUNet 3D/160x128x32': [0.07198143005371094, 0.055013902485370636, 0.05309624597430229], 'nnUNet 3D/160x128x32 (CLIP Pre-&Post-IVL)': [0.060034286230802536, 0.05875031650066376, 0.049891747534275055], 'nnUNet 3D/160x128x32 (LaW OCT)': [0.06659381836652756, 0.05495190992951393, 0.05585246533155441]}

5
{'nnUNet 3D/160x128x32': [0.08301436901092529, 0.06917698681354523, 0.11636274307966232], 'nnUNe